#Lid driven cavity

$$
\newcommand{\DdQq}[2]{{\mathrm D}_{#1}{\mathrm Q}_{#2}}
\newcommand{\drondt}{\partial_t}
\newcommand{\drondx}{\partial_x}
\newcommand{\drondy}{\partial_y}
\newcommand{\drondtt}{\partial_{tt}}
\newcommand{\drondxx}{\partial_{xx}}
\newcommand{\drondyy}{\partial_{yy}}
\newcommand{\dx}{\Delta x}
\newcommand{\dt}{\Delta t}
\newcommand{\grandO}{{\mathcal O}}
\newcommand{\density}[2]{\,f_{#1}^{#2}}
\newcommand{\fk}[1]{\density{#1}{\vphantom{\star}}}
\newcommand{\fks}[1]{\density{#1}{\star}}
\newcommand{\moment}[2]{\,m_{#1}^{#2}}
\newcommand{\mk}[1]{\moment{#1}{\vphantom{\star}}}
\newcommand{\mke}[1]{\moment{#1}{e}}
\newcommand{\mks}[1]{\moment{#1}{\star}}
$$

In this pratical work session, we consider the classical $\DdQq{2}{9}$ and $\DdQq{3}{15}$ to simulate a lid driven acvity modeling by the Navier-Stokes equations. The $\DdQq{2}{9}$ is used in dimension $2$ and the $\DdQq{3}{15}$ in dimension $3$.

##The $\DdQq{2}{9}$ for Navier-Stokes

The $\DdQq{2}{9}$ is defined by:
* a space step $\dx$ and a time step $\dt$ related to the scheme velocity $\lambda$ by the relation $\lambda=\dx/\dt$,
* nine velocities $\{(0,0), (\pm1,0), (0,\pm1), (\pm1, \pm1)\}$, identified in pyLBM by the numbers $0$ to $8$,
* nine polynomials used to build the moments
$$ \{1, \lambda X, \lambda Y, 3E-4, (9E^2-21E+8)/2, 3XE-5X, 3YE-5Y,X^2-Y^2, XY\},$$
where $E = X^2+Y^2$.
* three conserved moments $\rho$, $q_x$, and $q_y$,
* nine relaxation parameters (three are $0$ corresponding to conserved moments): $\{0,0,0,s_\mu,s_\mu,s_\eta,s_\eta,s_\eta,s_\eta\}$, where $s_\mu$ and $s_\eta$ are in $(0,2)$,
* equilibrium value of the non conserved moments
\begin{align}\mke{3} &= -2\rho + 3(q_x^2+q_y^2)/(\rho_0\lambda^2), \\ \mke{4} &= \rho+3(q_x^2+q_y^2)/(2\rho_0\lambda^2), \\ \mke{5} &= -q_x/\lambda, \\ \mke{6} &= -q_y/\lambda, \\ \mke{7} &= (q_x^2-q_y^2)/(\rho_0\lambda^2), \\ \mke{8} &= q_xq_y/(\rho_0\lambda^2),\end{align}
where $\rho_0$ is a given scalar.

This scheme is consistant at second order with the following equations (taken $\rho_0=1$)
\begin{gather*} \drondt\rho + \drondx q_x + \drondy q_y = 0,\\ \drondt q_x + \drondx (q_x^2+p) + \drondy (q_xq_y) = \mu \drondx (\drondx q_x + \drondy q_y ) + \eta (\drondxx+\drondyy)q_x, \\ \drondt q_y + \drondx (q_xq_y) + \drondy (q_y^2+p) = \mu \drondy (\drondx q_x + \drondy q_y ) + \eta (\drondxx+\drondyy)q_y,\end{gather*}
with $p=\rho\lambda^2/3$.

####Question 1

Propose a dictionary for a simulation of the Navier-Stokes equations on $(0,1)^2$.

####Question 2

In order to impose the boundary conditions, use the bounce-back conditions to fix $q_x=q_y=0$ at south, east, and west and $q_x=\rho u$, $q_y=0$ at north. The driven velocity $u$ could be $u=\lambda/10$.

####Question 3

The solution is governed by the Reynolds number $Re = \rho_0u / \eta$. Fix the relaxation parameters to have $Re=1000$. The relaxation parameters related to the bulk viscosity $\mu$ should be large enough to ensure the stability (for instance $\mu=10^{-3}$).

####Question 4

Compute the stationary solution of the problem obtained for large enough final time. Plot the solution with the function quiver of matplotlib. Test the stability of the scheme when the Reynolds number increases.

##The $\DdQq{3}{15}$ for Navier-Stokes

The $\DdQq{3}{15}$ is defined by:
* a space step $\dx$ and a time step $\dt$ related to the scheme velocity $\lambda$ by the relation $\lambda=\dx/\dt$,
* fifteen velocities $\{(0,0,0), (\pm1,0,0), (0,\pm1,0), (0,0,\pm1), (\pm1, \pm1,\pm1)\}$, identified in pyLBM by the numbers $\{0,\ldots,6,19,\ldots,26\}$,
* fifteen polynomials used to build the moments
$$ \{1, E-2, (15E^2-55E+32)/2, X, X(5E-13)/2, Y, Y(5E-13)/2, Z, Z(5E-13)/2, 3X^2-E, Y^2-Z^2, XY, YZ, ZX, XYZ \},$$
where $E = X^2+Y^2+Z^2$.
* four conserved moments $\rho$, $q_x$, $q_y$, and $q_z$,
* fifteen relaxation parameters (four are $0$ corresponding to conserved moments): $\{0, s_1, s_2, 0, s_4, 0, s_4, 0, s_4, s_9, s_9, s_{11}, s_{11}, s_{11}, s_{14}\}$,
* equilibrium value of the non conserved moments
\begin{aligned} \mke{1} &= -\rho + q_x^2 + q_y^2 + q_z^2,\\ \mke{2} &= -\rho,\\ \mke{4} &= -7q_x/3, \\ \mke{6} &= -7q_y/3, \\ \mke{8} &= -7q_z/3, \\ \mke{9} &= (2q_x^2-(q_y^2+q_z^2))/3, \\ \mke{10} &= q_y^2-q_z^2, \\ \mke{11} &= q_xq_y, \\ \mke{12} &= q_yq_z, \\ \mke{13} &= q_zq_x, \\ \mke{14} &= 0. \end{aligned}

This scheme is consistant at second order with the Navier-Stokes equations with the shear viscosity $\eta$ and the relaxation parameter $s_9$ linked by the relation
$$ s_9 = \frac{2}{1 + 6\eta /\dx}.$$

####Question 1

Propose a dictionary for a simulation of the Navier-Stokes equations on $(0,1)^3$.

####Question 2

In order to impose the boundary conditions, use the bounce-back conditions to fix $q_x=q_y=q_z=0$ at south, north, east, west, and bottom and $q_x=\rho u$, $q_y=q_z=0$ at top. The driven velocity $u$ could be $u=\lambda/10$.

####Question 3

Compute the stationary solution of the problem obtained for large enough final time. Plot the solution with the function quiver of matplotlib. Test the stability of the scheme when the Reynolds number increases.

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()